<img src="https://www.colorado.edu/rc/sites/default/files/page/logo.png"
     alt="Logo for Research Computing @ University of Colorado Boulder"
     width="400" />
# Parallel computation of $\pi$

In this example, each engine computes a unique estimate of pi.
The result is averaged across all engines in the cluster.

In [1]:
import ipyparallel
import random
import time
import numpy as np

def estimate_pi(n):
    count = 0
    for i in range(n):
        x = random.random()
        y = random.random()
        if (x**2 + y**2) <= 1:
            count += 1
    return 4.0*count/float(n)

## Serial calculation

In [2]:
print('\n\n\n')
print('Serial Estimation of Pi')
print('')

for i in range(8):
    nx = 10**i

    t0 = time.time()

    est_pi = estimate_pi(nx)

    t1 = time.time()
    tval = t1-t0

    msg = 'Estimation based on '+str(10**i)+' points: '
    tmsg = 'Calculation time (seconds) : '
    print(msg,est_pi,tmsg,tval)





Serial Estimation of Pi

Estimation based on 1 points:  4.0 Calculation time (seconds) :  1.5974044799804688e-05
Estimation based on 10 points:  2.8 Calculation time (seconds) :  1.0967254638671875e-05
Estimation based on 100 points:  2.88 Calculation time (seconds) :  5.1975250244140625e-05
Estimation based on 1000 points:  3.156 Calculation time (seconds) :  0.0005199909210205078
Estimation based on 10000 points:  3.1312 Calculation time (seconds) :  0.004929065704345703
Estimation based on 100000 points:  3.15036 Calculation time (seconds) :  0.04758930206298828
Estimation based on 1000000 points:  3.145016 Calculation time (seconds) :  0.40417909622192383
Estimation based on 10000000 points:  3.141808 Calculation time (seconds) :  4.154988765716553


## Parallel calculation

In [3]:
rc=ipyparallel.Client(profile='default')
nengines = len(rc)
all_proc = rc[:]
all_proc.block = True

### Import modules on the engines

Each engine has its own namespace, so functions, variables ... need to be defined on those engines. We can do this with the parallel magic commands as show. Alternativley you can use `sync_imports`

In [4]:
with all_proc.sync_imports():
    import random

importing random on engine(s)


### Running the calculation in parallel

In [5]:
print('\n\n\n')
print('Parallel Estimation of Pi')
print('')

#Now in parallel
for i in range(2,8):
    nx = 10**i

    t0 = time.time()

    #[nx//nengines]*nengines creates a list of length nengines, where each element
    # has value nx//nengines.  Each process gets one element of this list 
    # and passes it to estimate_pi

    pi_estimates = all_proc.map_sync(estimate_pi, [nx//nengines]*nengines)

    est_pi = np.mean(pi_estimates)

    t1 = time.time()
    tval = t1-t0

    msg = 'Estimation based on '+str(10**i)+' points: '
    tmsg = 'Calculation time (seconds) : '
    print(msg,est_pi,tmsg,tval)







Parallel Estimation of Pi

Estimation based on 100 points:  3.24 Calculation time (seconds) :  0.021104812622070312
Estimation based on 1000 points:  3.164 Calculation time (seconds) :  0.025281906127929688
Estimation based on 10000 points:  3.1464 Calculation time (seconds) :  0.017162561416625977
Estimation based on 100000 points:  3.14264 Calculation time (seconds) :  0.029888153076171875
Estimation based on 1000000 points:  3.14184 Calculation time (seconds) :  0.15157294273376465
Estimation based on 10000000 points:  3.1417976 Calculation time (seconds) :  1.1393918991088867
